In [1]:
import os
import openai #needed for error handling
from openai import OpenAI

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [2]:
client = OpenAI(
  api_key=os.environ['OPENAI_API_KEY']  
)

In [3]:
TEXT_MODEL = "gpt-3.5-turbo"

#sets the persona for the AI assistant using a system message
context = [{'role':'system', 'content':"""You are a friendly AI assistant that 
                                              helps compose professional-sounding tweets 
                                              for Twitter that often go viral based on a 
                                              website I provide. You will provide a summary 
                                              of the website in 50 words or less."""
            }]  

In [4]:
# Each interaction with the AI assistant is a new session so the entire chat/message history, 
# including user prompts and assistant responses must be included in each exchange with the
# model/assistant so that it "remembers"

def collect_messages(role, message): #keeps track of the message exchange between user and assistant
    context.append({'role': role, 'content':f"{message}"})

In [5]:
# Sends the prompts to the model for a completion/response

def get_completion(temperature=0): 
    try:
        response = client.chat.completions.create(
            model=TEXT_MODEL,
            messages=context, 
            temperature=temperature, 
        )

        print("\n Assistant: ", response.choices[0].message.content, "\n")

        return response.choices[0].message.content
    except openai.APIError as e:
        print(e.http_status)
        print(e.error)
        return e.error

In [6]:
# Create images from scratch based on the website summary

def generate_image(summary):
    print(summary)
    
    try:
        response = client.images.generate(
          model="dall-e-3",
          prompt=summary,
          size="1024x1024",
          quality="standard",
          n=1, #select the number of images you want generated
        )
        
        image_url = response.data[0].url #URLs will expire after an hour

        return image_url
    except openai.APIError as e:
        print(e.http_status)
        print(e.error)
        return e.error

In [ ]:
#Start the conversation between the user and the AI assistant/chatbot
    
while True:
    summary = get_completion()
    
    collect_messages('assistant', summary) #stores the response from the AI assistant
    
    user_prompt = input('User: ') #input box for entering prompt
    
    if user_prompt == 'exit': #end the conversation with the AI assistant
        print("\n Goodbye")
        break

    collect_messages('user', user_prompt) #stores the user prompt

In [ ]:
url = generate_image(summary)

In [ ]:
from IPython.display import Image
Image(url=url)